After completing the Lab you will:  
- Create a python program that asks the user for weather in a zip code
- The program will convert that to Long/Lat (there are websites/libraries/apis for this)
- Scrape the 7 day forecast and display to the user
- Display the current conditions - humidity, windspeed, Dewpoint and Last Update Time
- Store the 7 day forecast in a MongoDB
- Store the current conditions in a MongoDB



group: Avery Schebell (ajs6av); Amshala Bharathan (ab5sz); Mianchen Cao (mc7sq)"

In [69]:
import pandas as pd

- Create a python program that asks the user for weather in a zip code

In [176]:
zipcode = input("Enter Zip Code: ")
print("Zipcode: " + zipcode)

Enter Zip Code: 20171
Zipcode: 20171


- The program will convert that to Long/Lat (there are websites/libraries/apis for this)

In [177]:
# https://pypi.org/project/pgeocode/
# found this python library online that can give information based on country and zipcode 
import pgeocode
nomi = pgeocode.Nominatim('us') #country code for US
placename = str(nomi.query_postal_code(zipcode)[5])
statename = str(nomi.query_postal_code(zipcode)[3])
latitude = str(nomi.query_postal_code(zipcode)[9])  
longitude = str(nomi.query_postal_code(zipcode)[10])  
coordinates = (latitude, longitude)
location = (placename + ", " + statename)
print(location)
print(coordinates)

Fairfax, Virginia
('38.9252', '-77.3928')


- Scrape the 7 day forecast and display to the user

In [178]:
import requests
from bs4 import BeautifulSoup
page = requests.get("https://forecast.weather.gov/MapClick.php?lat="+latitude+"&lon="+longitude)
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
#print(tonight.prettify())

In [179]:
period = tonight.find(class_="period-name").get_text() 
short_desc = tonight.find(class_="short-desc").get_text() 
temp = tonight.find(class_="temp").get_text() 
#print(period)
#print(short_desc) 
#print(temp)

In [180]:
img = tonight.find("img") 
desc = img['title'] 
#print(desc)

In [181]:
period_tags = seven_day.select(".tombstone-container .period-name") 
periods = [pt.get_text() for pt in period_tags]
#periods

In [182]:
short_descs = [sd.get_text() for sd in seven_day.select('.tombstone-container .short-desc')]
temps = [t.get_text() for t in seven_day.select('.tombstone-container .temp')]
descs = [d['title'] for d in seven_day.select('.tombstone-container img')]
#print(short_descs,temps,descs)

In [183]:
weatherdf = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc": desc
})
weatherdf

,period,short_desc,temp,desc
0,Tonight,Mostly Clear,Low: 31 °F,"Tonight: Mostly clear, with a low around 31. N..."
1,Monday,Sunny,High: 47 °F,"Tonight: Mostly clear, with a low around 31. N..."
2,MondayNight,Mostly Clear,Low: 30 °F,"Tonight: Mostly clear, with a low around 31. N..."
3,Tuesday,Slight ChanceRain thenRain,High: 46 °F,"Tonight: Mostly clear, with a low around 31. N..."
4,TuesdayNight,Rain,Low: 39 °F,"Tonight: Mostly clear, with a low around 31. N..."
5,Wednesday,Partly Sunny,High: 51 °F,"Tonight: Mostly clear, with a low around 31. N..."
6,WednesdayNight,Mostly Clear,Low: 32 °F,"Tonight: Mostly clear, with a low around 31. N..."
7,Thursday,Sunny,High: 44 °F,"Tonight: Mostly clear, with a low around 31. N..."
8,ThursdayNight,Mostly Clear,Low: 26 °F,"Tonight: Mostly clear, with a low around 31. N..."


In [184]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False) 
weather["temp_num"] = temp_nums.astype('int')
#temp_nums

In [185]:
#weather["temp_num"].mean()

In [186]:
is_night = weather["temp"].str.contains("Low") 
weather["is_night"] = is_night
is_night
weather[is_night]

,period,short_desc,temp,desc,temp_num,is_night
1,Tonight,Clear,Low: 30 °F,"This Afternoon: Sunny, with a high near 55. No...",30,True
3,MondayNight,Partly Cloudy,Low: 32 °F,"This Afternoon: Sunny, with a high near 55. No...",32,True
5,TuesdayNight,Rain,Low: 42 °F,"This Afternoon: Sunny, with a high near 55. No...",42,True
7,WednesdayNight,Partly Cloudy,Low: 34 °F,"This Afternoon: Sunny, with a high near 55. No...",34,True


- Display the current conditions - humidity, windspeed, Dewpoint and Last Update Time

In [187]:
current_conditions = soup.find(id="current_conditions_detail")
#print(current_conditions)
soup = BeautifulSoup(page.content, 'html.parser')
humidity = soup.find_all('td')[1].get_text()
wind_speed = soup.find_all('td')[3].get_text()
dewpoint = soup.find_all('td')[7].get_text()
last_update_time = soup.find_all('td')[13].get_text().strip()
current_condition_display =pd.DataFrame([[location,humidity,wind_speed,dewpoint,last_update_time]], columns=['Location', 'Humidity','Wind Speed','Dewpoint','Last Update Time'])
current_condition_display

,Location,Humidity,Wind Speed,Dewpoint,Last Update Time
0,"Fairfax, Virginia",51%,W 5 mph,23°F (-5°C),13 Nov 7:52 pm EST


- Store the 7 day forecast in a MongoDB

In [188]:
#import quandl
import pymongo
from pymongo import MongoClient

In [189]:
# Making a Connection with MongoClient
client = MongoClient("mongodb://localhost:27017/")
# database
db = client["DS2002"]
# collection
company = db["Location"]

In [190]:
#db.list_collection_names()

In [191]:
#posts = db.
#post_id = posts.insert_one(post).inserted_id
#print("Document ID: ", post_id)

data_dict = weatherdf.to_dict()

In [192]:
#company.insert_one({"index":"Sensex","data":data_dict})

In [193]:
#db.lab5.insert_many(data_dict)

In [194]:
data_dict2 = current_condition_display. to_dict()
#db.lab5.insert_many(data_dict2)

In [108]:
weather.to_csv('weather.csv')
current_condition_display.to_csv('current_conditions.csv')